In [0]:
%run ../config/init_config


In [0]:
from pyspark.sql.functions import from_json, to_date, lit, col, row_number, broadcast, count, sum
from pyspark.sql.window import Window

country_path = "file:/Workspace/Users/joelledidanera@gmail.com/03_Databricks_Medallion/includes/country.json"

country_df = spark.read.json(country_path)

In [0]:
def process_customer_silver():
    customer_schema = "customer_id STRING, name STRING, age INT, country_code STRING"

    query = (spark.readStream
                .format("delta")
                .table("bronze_raw")
                .filter("source_type = 'customers'")
                .select(from_json("value", customer_schema).alias("v"), "processed_time")
                .select("v.*", "processed_time")
                .join(broadcast(country_df)).where(col("country_code") == country_df["code"])
                .writeStream
                .format("delta")
                .option("mergeSchema", "true")
                .option("checkpointLocation", silver_checkpoint_path + "/customers")
                .trigger(availableNow=True)
                .table("customer_silver")
                )
                        

    query.awaitTermination()

In [0]:
process_customer_silver()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS current_customer_silver (
  customer_id STRING,
  name STRING,
  age INT,
  country_code STRING,
  country_label STRING,
  expired_at TIMESTAMP,
  is_current BOOLEAN
) USING delta;


In [0]:
merge_script = """
MERGE INTO current_customer_silver t
USING (
    SELECT *, s.customer_id as merge_id FROM update_customer s
    UNION ALL
    SELECT s.*, NULL as merge_id FROM update_customer s
    JOIN current_customer_silver t ON s.customer_id = t.customer_id
    WHERE t.is_current = true
    ) as customer_updates
ON t.customer_id = customer_updates.merge_id AND t.is_current = true
WHEN MATCHED AND customer_updates.merge_id IS NULL THEN
  UPDATE SET *
WHEN MATCHED THEN
  UPDATE SET t.is_current = false, t.expired_at = current_timestamp()
WHEN NOT MATCHED THEN
  INSERT *
""" 
def process_each_customer_batch(batch_df, batch_id):
    window = Window.partitionBy("customer_id").orderBy(col("processed_time").desc())

    batch_df = (
        batch_df
        .withColumn("row_num", row_number().over(window))
        .filter(col("row_num") == 1)  # garde la plus récente
        .drop("row_num")
        .withColumn("expired_at", lit(None).cast("timestamp"))
        .withColumn("is_current", lit(True).cast("boolean"))
        .withColumnRenamed("label", "country_label")
        .dropDuplicates(['customer_id'])
      )
    batch_df.createOrReplaceTempView("update_customer")
    #display(batch_df)
    batch_df.sparkSession.sql(merge_script)


def process_current_customer_silver():
  query = (spark.readStream
              .table("customer_silver")
              .writeStream
              .foreachBatch(process_each_customer_batch)
              .option("mergeSchema", "true")
              .option("checkpointLocation", silver_checkpoint_path + "/current_customer")
              .trigger(availableNow=True)
              .start()
              )

  query.awaitTermination()

In [0]:
process_current_customer_silver()